In [ ]:
#company scraping project

In [1]:
 #! /bin/python3
from bs4 import BeautifulSoup as scraper
import re
import grequests

import threading
import time
import pandas as pd 
import numpy as np


threads = []
link = []
company_info = {'Name':[],'Phone_No':[],'Email':[]}
company_name = []
name = "#"
temp = []
not_available_company = []
generate_url_dict = {}

with open('company.txt','r') as file:
    while name:
        name = file.readline()
        if name not in company_name and len(name) != 0:
            company_name.append(name.strip())

link_re = ('.com/contact-us/|.net/|.net/|.gov/|.gov|.com|.com/')
#link_re = "(|.com/|.com|.net|.gov|.com.np|.com/about|.com/contact-us|.com/about|.com/about-us|.com/faq|.com/customer-center)$"
def scrape_web(page,search_name):
    temp_link = []
    soup = scraper(page.content,"html.parser")
    for links in soup.find_all('a', attrs ={'href':re.compile("^https://(www.|blog.)")}):
        link = links.get('href')
        if re.findall(link_re,link):
            nm = search_name.split()
            nm = nm[0].lower()
            strip = len('https://www.')+len(nm)+2
            if re.search(nm,link[:strip]):
                if link not in temp_link:
                    temp_link.append(link)
    
    if len(temp_link) != 0:
        print(search_name)
        page_requests(temp_link,search_name)
    #create_url_list(links,search_name)






def create_url_list(link,companyname):
    generate_url_dict[companyname].append(link)
        
    
    

def exception(request, exception):
        print(f"Problem: {request.url} : {exception}\n")

def page_requests(urls,companyname):
    results = grequests.map((grequests.get(u,timeout=6) for u in urls), exception_handler=exception, size=5)
    
    for page in results:
        if page is not None:
            if page.status_code == 200:
                print(f"✅ {page.url}")
                find_tel_email(page,companyname)
            else:
                if company not in not_available_company:
                    not_available_company.append(companyname)
                print(f"❌{page.url}")
            

    
def find_tel_email(page,companyname):
    tel = []
    email  = []
 
    soup = scraper(page.content,"html.parser")
    for info in soup.find_all('a', attrs = {'href': re.compile("^tel:")}):
        telephone = info.get('href')[4:]
        if telephone not in tel:
            tel.append(telephone)
    for info in soup.find_all('a',attrs = {'href': re.compile("^mailto:")}):
        mail = info.get('href')[7:]
        if mail not in email:
            email.append(mail)
            
    if len(tel) == 0 and len(email) == 0:
        if company not in not_available_company:
            not_available_company.append(companyname)
        return 
    if companyname not in company_info['Name']:
        company_info['Name'].append(companyname)
        if len(tel) == 0:
            tel = np.nan
        elif len(tel) == 1:
            tel = tel[0]
                
        if len(email) == 0:
            email = np.nan  # if email is null add Nan
        elif len(email) == 1:
            email = email[0] #if only single value in list unpack it 
                
        company_info['Phone_No'].append(tel)
        company_info['Email'].append(email)
        
        
    
        
seed = "https://search.yahoo.com/search?p="
yahoo_search_url = []
for company in company_name:
    yahoo_search_url.append(seed+company)

print('⏳ Searching company ...')
results = grequests.map((grequests.get(u,timeout=6) for u in yahoo_search_url), exception_handler=exception, size=7)
print('✅ Search complete finding email and phone ...')
for page,company in zip(results,company_name):
    if page is not None and page.status_code == 200:
        scrape_web(page,company)


    
    
    
print("\n EMAIL AND TELEPHONE NO\n\n")
df = pd.DataFrame(company_info)
display(df)
df.to_csv('company_scrape.csv',index = False)

print("\n NOT FOUND COMPANIES:\n")
print(not_available_company)


⏳ Searching company ...
✅ Search complete finding email and phone ...
Nepal Bank Ltd. (NBL)
✅ https://www.nepalbank.com.np/
✅ https://www.nepalbank.com.np/personal/products/saving-account
✅ https://www.nepalbank.com.np/personal/retail-loan
✅ https://www.nepalbank.com.np/personal/services/atm-debit-card
✅ https://www.nepalbank.com.np/personal/nbl-web-remit
✅ https://www.nepalbank.com.np/personal/interest-rate/interest-rates-on-deposits
✅ https://www.nepalbank.com.np/personal/services/safe-deposit-locker
Nabil Bank Ltd. (NABIL)
✅ https://www.nabilbank.com/en/
✅ https://www.nabilbank.com/en/personal/card-e-banking/products/nabil-icard
✅ https://www.nabilbank.com.np/
Himalayan Bank Ltd. (HBL)
✅ https://www.himalayanbank.com/en/
✅ https://www.himalayanbank.com/en/branches-atms
✅ https://www.himalayanbank.com/en/contact-us
✅ https://www.himalayanbank.com/en/loan-products
✅ https://www.himalayanbank.com/en/card-services
✅ https://www.himalayanbank.com/en/ancillary-products
✅ https://www.himal

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


✅ https://www.sunrisebank.com.np/business-solution/policies-and-guidelines
✅ https://www.sunrisebank.com.np/branchless-banking
✅ https://www.sunrisebank.com.np/extension-counter
✅ https://www.sunrisebank.com.np/staging/branches
✅ https://www.sunrisebank.com.np/
NMB Bank Ltd. (NMB)
✅ https://www.nmb.com.np/
✅ https://www.nmb.com.np/about-us/introduction
✅ https://www.nmbbank.net/login.xhtml?faces-redirect=true
Prabhu Bank Ltd. (PRABHU)
✅ https://www.prabhubank.com/electronic-banking/-internet-banking
✅ https://www.prabhubank.com/profile/profile-of-bank
✅ https://www.prabhubank.com/
Mega Bank Nepal Ltd. (Mega)
✅ https://www.megabanknepal.com/
✅ https://www.megabanknepal.com/tick-banking
✅ https://www.megabanknepal.com/online-account
✅ https://www.megabanknepal.com/kyc-search
✅ https://www.megabanknepal.com/bank-guarantee-search
✅ https://www.megabanknepal.com/video-library
✅ https://www.megabanknepal.com/downloads
✅ https://www.megabanknepal.com/career
✅ https://www.megabanknepal.com/dow

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


✅ https://www.corporatebank.com.np/deposit/
✅ https://www.corporatebank.com.np/loan/
✅ https://www.corporatebank.com.np/contact-us/
Sindhu Bikas Bank Ltd.
✅ https://www.sindhubank.com.np/
✅ https://www.sindhubank.com.np/introduction/
✅ https://www.sindhubank.com.np/product_category/deposits-savings/
✅ https://www.sindhubank.com.np/services/mobile-banking/
✅ https://www.sindhubank.com.np/rates/interest-rate/
✅ https://www.sindhubank.com.np/report_type/quarterly/
✅ https://www.sindhubank.com.np/category/general-notice/
✅ https://www.sindhubank.com.np/branches/
✅ https://www.sindhubank.com.np/news/
✅ https://www.sindhubank.com.np/team/
✅ https://www.sindhubank.com.np/contact-us/
✅ https://www.sindhubank.com.np/bod/
Green Development Bank Ltd.
❌https://www.greenbank.com.np/
Garima Bikas Bank Ltd.
✅ https://www.garimabank.com.np/
✅ https://www.garimabank.com.np/#!
✅ https://www.garimabank.com.np/safe-deposit-locker
✅ https://www.garimabank.com.np/products/sunaulo-bhabisya-nikchhyap-yojana
✅

...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


✅ https://www.nadeplaghubitta.com/disclaimer.html
✅ https://www.nadeplaghubitta.com/deposit.html
Ghodighoda Laghubitta Bittiya Sanstha Limited
Problem: https://www.ghodighoda.com.np/ : HTTPSConnectionPool(host='www.ghodighoda.com.np', port=443): Max retries exceeded with url: / (Caused by SSLError(CertificateError("hostname 'www.ghodighoda.com.np' doesn't match 'ghodighoda.com.np'")))

Problem: https://www.ghodighoda.com.np/content/information : HTTPSConnectionPool(host='www.ghodighoda.com.np', port=443): Max retries exceeded with url: /content/information (Caused by SSLError(CertificateError("hostname 'www.ghodighoda.com.np' doesn't match 'ghodighoda.com.np'")))

Problem: https://www.ghodighoda.com.np/content/saving : HTTPSConnectionPool(host='www.ghodighoda.com.np', port=443): Max retries exceeded with url: /content/saving (Caused by SSLError(CertificateError("hostname 'www.ghodighoda.com.np' doesn't match 'ghodighoda.com.np'")))

Problem: https://www.ghodighoda.com.np/content/mobile

,Name,Phone_No,Email
0,Nepal Bank Ltd. (NBL),"[014247999, 014239888, 01-4222365]",NaN
1,Himalayan Bank Ltd. (HBL),"[014227749, 01-4227749, 01-4246218, 01-4246219...","[himb@himalayanbank.com, sunil.gorkhali@himala..."
2,Nepal SBI Bank Ltd. (NSBI),"[01-4620780, +977 9803547429, +977 9818560497]",NaN
3,NIC ASIA Bank Ltd. (NIC),"[1660-01-77771, 9801227755]",NaN
4,Kumari Bank Ltd. (Kumari),"[01-4443077, 01-4443078, 01-4443079]",communication@kumaribank.com
5,Laxmi Bank Ltd. (Laxmi),"[014444684, 014444685]",info@laxmibank.com
6,Siddhartha Bank Ltd. (SBL),+977-01-5970020,customercare@sbl.com.np
7,Prime Commercial Bank Ltd. (Prime),NaN,"[information.officer@pcbl.com.np, prashant.dan..."
8,Sunrise Bank Ltd. (Sunrise),"[16600122444, 4004560, 4004561, 4004562]",info@sunrisebank.com.np
9,NMB Bank Ltd. (NMB),+97715970150,call@nmb.com.np



 NOT FOUND COMPANIES:

['Nabil Bank Ltd. (NABIL)', 'Prime Commercial Bank Ltd. (Prime)', 'Sunrise Bank Ltd. (Sunrise)', 'NMB Bank Ltd. (NMB)', 'Civil Bank Ltd. (Civil)', 'Miteri Development Bank Ltd.', 'Muktinath Bikas Bank Ltd.', 'Corporate Development Bank Ltd.', 'Green Development Bank Ltd.', 'Kamana Sewa Bikas Bank Ltd.', 'Tinau Mission Bikas Bank Ltd.', 'Gurkhas Finance Ltd.', 'Goodwill Finance Ltd.', 'United Finance Co. Ltd. United', 'Capital Merchant Banking & Finance Co. Ltd.', 'Crystal Finance Ltd.', 'Manjushree Financial Institution Ltd.', 'Reliance Finance Ltd.', 'Nirdhan Utthan Laghubitta Bittiya Sanstha Limited', 'Deprosc Laghubitta Bittiya Sanstha Limited', 'Summit Laghubitta Bittiya Sanstha Ltd.', 'Mirmire Microfinance Development Bank Limited', 'Mahila Sahayatra Microfinance Bittiya Sanstha Ltd.', 'FORWARD Community Microfinance Bittiya Sanstha Ltd.', 'NADEP Laghubitta Bittiya Sanstha Limited', 'Asha Laghubitta Bittiya Sanstha Ltd', 'Ganapati Microfinance Bittiya Sasth

C:\Users\Dell\Jupyter LAB\venv\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
